Name: Saral Gautam
Student ID: 30618428
email: sgau0007@student.monash.edu

# FIT3182 Assignment 1: Part A

Task 1:
The data model will follow the document data model. This is because the data contains multiple different types for example datetime (in hotspot_historic), date, integers and strings (used in preciptation measurements). Additionally the data is relatively unstructured- in climate historic each set of measurements is taken daily without any kind of grouping between the measurements and in the hotspot data there is no sorting with any of the dates or datetimes. Furthermore since the dates are common between both the hotspot_historic and climate_historic we can group the documents together and have overall denormalised data. This is optimal since we only plan to run search queries through the data which will be much faster when the documents are grouped together in this way. The document data model also allows for some attributes to be empty and this may be the case since one climate data may have zero or many fire data, hence leading to some possibly empty attributes.
Example of data:

In [62]:
from pprint import pprint
import pymongo
from pymongo import MongoClient
import csv

In [63]:
def samedate(hotspot_date, climate_date):
    # this function checks if two dates are the same
    # This function was made because date is forrmatted differently in hotspot_historic and climate_historic
    # eg. hotspot historic '27/12/2018' and climate historic '27/12/18'
    actual_date = []
    final_date = []
    join_string = ''
    date_length = len(hotspot_date)
    for k in range(date_length - 1, -1, -1):
        if k == 7 or k == 6:
            continue
        else:
            actual_date.append(hotspot_date[k])
    for j in range(date_length - 3, -1, -1):
        final_date.append(actual_date[j])

    return (join_string.join(final_date) == climate_date)

In [64]:
# Task 2
# Question 1
# So we have to load all the data from the csv files into a new database fit3182_assignment_db
# Check if the main collection already exists
try:
    the_collection.drop()
except NameError:
    pass
# Now create the data base
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
with open('climate_historic.csv', 'r') as file_one:  # open climate historic file
    # now create a csv object for each of the files
    climate_data = csv.reader(file_one)
    climate_lines = []
    count = 0
    for c_line in climate_data:
        if count == 0:
            count += 1
        else:
            climate_lines.append(c_line)   # put each line of the data in a list 
            

            
with open('hotspot_historic.csv', 'r') as file_two: # open hotspot historic file
    hotspot_data = csv.reader(file_two)
    hotspot_lines = []
    count = 0
    for h_line in hotspot_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            hotspot_lines.append(h_line)  # put each line of the data in a list

documents = []
for climate_line in climate_lines:
    matched = False
    for hotspot_line in hotspot_lines:
        if samedate(hotspot_line[4], climate_line[1]):
            new_entry = {
             "latitude" : float(hotspot_line[0]),
             "longitude": float(hotspot_line[1]),
             "datetime" : hotspot_line[2],
             "confidence" : float(hotspot_line[3]),
             "surface_temperature_celsius" : float(hotspot_line[5]),
            "station": float(climate_line[0]),
            "date": climate_line[1],
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation": climate_line[6],
            "GHI_w/m2": float(climate_line[7])}
            documents.append(new_entry)
            matched = True
    if matched == False: # there was not a fire on this date
        # Set None's for all the unmatchable values cause there was no fire on this date
        no_fire = {
            "latitude" : None,
            "longitude": None,
            "datetime" : None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "station": float(climate_line[0]),
            "date": climate_line[1],
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation": climate_line[6],
            "GHI_w/m2": float(climate_line[7])
        }
        documents.append(no_fire)


result = the_collection.insert_many(documents)  
# cursor = the_collection.find({})
# for item in cursor:
#     pprint(item)


Below are all the queries for Task 2 Question 2

In [75]:
# a)
pprint(the_collection.find_one({"date": "12/12/18"}, 
                               {"date": 1, "air_temperature_celsius" : 1, "relative_humidity": 1, "windspeed_knots": 1,
                                "max_wind_speed": 1, "precipitation": 1, "GHI_w/m2": 1
                               }))

{'GHI_w/m2': 156.0,
 '_id': ObjectId('609f209d8c1be29d705b42c1'),
 'air_temperature_celsius': 19.0,
 'date': '12/12/18',
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 55.3,
 'windspeed_knots': 6.2}


In [74]:
# b) 
answer = the_collection.find({"$and": [
    {"surface_temperature_celsius": {"$gte": 65}},
    {"surface_temperature_celsius": {"$lte": 100}}   
]},  
{"latitude": 1, "longitude": 1, "surface_temperature_celsius": 1, "confidence": 1}
                             
                             
)

for item in answer:
    pprint(item)

{'_id': ObjectId('609f209d8c1be29d705b4231'),
 'confidence': 96.0,
 'latitude': -38.127,
 'longitude': 143.82,
 'surface_temperature_celsius': 77.0}
{'_id': ObjectId('609f209d8c1be29d705b4237'),
 'confidence': 100.0,
 'latitude': -37.294,
 'longitude': 141.232,
 'surface_temperature_celsius': 87.0}
{'_id': ObjectId('609f209d8c1be29d705b4238'),
 'confidence': 99.0,
 'latitude': -37.293,
 'longitude': 141.245,
 'surface_temperature_celsius': 85.0}
{'_id': ObjectId('609f209d8c1be29d705b423d'),
 'confidence': 91.0,
 'latitude': -37.247,
 'longitude': 141.278,
 'surface_temperature_celsius': 68.0}
{'_id': ObjectId('609f209d8c1be29d705b424d'),
 'confidence': 92.0,
 'latitude': -36.098,
 'longitude': 143.74,
 'surface_temperature_celsius': 67.0}
{'_id': ObjectId('609f209d8c1be29d705b4272'),
 'confidence': 89.0,
 'latitude': -36.94,
 'longitude': 143.281,
 'surface_temperature_celsius': 69.0}
{'_id': ObjectId('609f209d8c1be29d705b4278'),
 'confidence': 87.0,
 'latitude': -37.862,
 'longitude':

In [81]:
# c)
fifteenth = the_collection.find({"date": '15/12/18'}, {"date": 1, "surface_temperature_celsius": 1, "air_temperature_celsius": 1, 
                             "relative_humidity": 1, "max_wind_speed": 1
                             })
for item in fifteenth:
    pprint(item)

sixteenth = the_collection.find({"date": '16/12/18'}, {"date": 1, "surface_temperature_celsius": 1, "air_temperature_celsius": 1, 
                             "relative_humidity": 1, "max_wind_speed": 1
                             })
for item in sixteenth:
    pprint(item)


{'_id': ObjectId('609f209d8c1be29d705b42c4'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 42.0}
{'_id': ObjectId('609f209d8c1be29d705b42c5'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 36.0}
{'_id': ObjectId('609f209d8c1be29d705b42c6'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 38.0}
{'_id': ObjectId('609f209d8c1be29d705b42c7'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 40.0}
{'_id': ObjectId('609f209d8c1be29d705b42c8'),
 'air_temperature_celsius': 18.0,
 'date': '16/12/18',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celsius': 43.0}
{'_id': ObjectId('609f209d8c1be29d705b42

In [82]:
# d)
d_answer = the_collection.find({"$and": [
    {"confidence": {"$gte": 80}},
    {"confidence": {"$lte": 100}}   
]},  
{"datetime": 1, "air_temperature_celsius": 1, "surface_temperature_celsius": 1, "confidence": 1}
                             
                             
)
for item in d_answer:
    pprint(item)

{'_id': ObjectId('609f209d8c1be29d705b4231'),
 'air_temperature_celsius': 13.0,
 'confidence': 96.0,
 'datetime': '2018-10-10T04:04:19',
 'surface_temperature_celsius': 77.0}
{'_id': ObjectId('609f209d8c1be29d705b4237'),
 'air_temperature_celsius': 13.0,
 'confidence': 100.0,
 'datetime': '2018-10-15T04:23:01',
 'surface_temperature_celsius': 87.0}
{'_id': ObjectId('609f209d8c1be29d705b4238'),
 'air_temperature_celsius': 13.0,
 'confidence': 99.0,
 'datetime': '2018-10-15T04:23:01',
 'surface_temperature_celsius': 85.0}
{'_id': ObjectId('609f209d8c1be29d705b423b'),
 'air_temperature_celsius': 22.0,
 'confidence': 94.0,
 'datetime': '2018-10-17T13:36:12',
 'surface_temperature_celsius': 43.0}
{'_id': ObjectId('609f209d8c1be29d705b423d'),
 'air_temperature_celsius': 22.0,
 'confidence': 91.0,
 'datetime': '2018-10-17T04:10:45',
 'surface_temperature_celsius': 68.0}
{'_id': ObjectId('609f209d8c1be29d705b423f'),
 'air_temperature_celsius': 22.0,
 'confidence': 82.0,
 'datetime': '2018-10-1

In [98]:
# e)
e_answer = the_collection.find().sort("surface_temperature_celsius", pymongo.DESCENDING).limit(10)
for item in e_answer:
    pprint(item)

{'GHI_w/m2': 111.0,
 '_id': ObjectId('609f209d8c1be29d705b4237'),
 'air_temperature_celsius': 13.0,
 'confidence': 100.0,
 'date': '15/10/18',
 'datetime': '2018-10-15T04:23:01',
 'latitude': -37.294,
 'longitude': 141.232,
 'max_wind_speed': 13.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 50.7,
 'station': 948702.0,
 'surface_temperature_celsius': 87.0,
 'windspeed_knots': 6.1}
{'GHI_w/m2': 227.0,
 '_id': ObjectId('609f209d8c1be29d705b4292'),
 'air_temperature_celsius': 28.0,
 'confidence': 100.0,
 'date': '29/11/18',
 'datetime': '2018-11-29T03:52:10',
 'latitude': -37.606,
 'longitude': 149.312,
 'max_wind_speed': 16.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.7,
 'station': 948702.0,
 'surface_temperature_celsius': 87.0,
 'windspeed_knots': 9.3}
{'GHI_w/m2': 202.0,
 '_id': ObjectId('609f209d8c1be29d705b42aa'),
 'air_temperature_celsius': 26.0,
 'confidence': 99.0,
 'date': '30/11/18',
 'datetime': '2018-11-30T04:34:57',
 'latitude': -37.6,
 'longitude': 149.325,

In [107]:
# f)
f_answer = the_collection.find({"latitude" : {"$ne": None}}).toArray()

# aggregate([{"$group":{"_id":"date", "count":{"$sum":1}}}])


AttributeError: 'Cursor' object has no attribute 'toArray'